## CRAWL dari REPUBLIKA

### Def untuk crawling dari Portal REPUBLIKA
Note : 
- ganti User-Agent pada headers -> ke user agent browser sendiri

In [ ]:
import os
import time
import csv
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote # Penting untuk keyword dengan spasi

def republika_scraper(keywords, n_berita):
    # Menggunakan session untuk mengelola koneksi dan header
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
        'Referer': 'https://republika.co.id/'
    })

    save_dir = r"/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/corpus/republika"
    os.makedirs(save_dir, exist_ok=True)

    # === PERUBAHAN UTAMA: Loop untuk setiap keyword ===
    for keyword in keywords:
        print(f"\n🚀 Mulai scraping untuk kata kunci: '{keyword}'")
        
        data_republika = []
        page = 1

        while len(data_republika) < n_berita:
            # Menggunakan API endpoint yang benar untuk hasil yang reliable
            api_url = f'https://republika.co.id/search/v3/q?q={quote(keyword)}&sortby=time&page={page}'
            
            try:
                print(f"🔄 Mengambil data dari API halaman {page} untuk '{keyword}'...")
                response = session.get(api_url, timeout=15)
                response.raise_for_status() # Cek jika ada error HTTP
                search_results = response.json()
                articles = search_results.get('articles', [])

            except requests.exceptions.RequestException as e:
                print(f"❌ Gagal mengakses API: {e}")
                break
            except requests.exceptions.JSONDecodeError:
                print(f"❌ Gagal parsing JSON. Kemungkinan diblokir atau halaman kosong.")
                break

            if not articles:
                print("✅ Tidak ada artikel lagi dari API.")
                break

            for article_data in articles:
                try:
                    url = article_data.get('url')
                    judul = article_data.get('title', 'Tanpa Judul')
                    
                    if not url:
                        continue

                    # Mengambil halaman detail artikel
                    detail_res = session.get(url, timeout=10)
                    detail_soup = BeautifulSoup(detail_res.content, 'html.parser')
                    
                    # Menggunakan class selector yang benar untuk konten berita
                    konten = detail_soup.find('div', class_='post-content')
                    
                    if konten:
                        # Membersihkan teks dari "Baca Juga"
                        for tag in konten.find_all(['strong', 'em'], text=lambda t: t and 'baca juga' in t.lower()):
                            tag.find_parent('p').decompose()
                        isi = konten.get_text(separator=' ', strip=True)
                    else:
                        isi = "Tidak ditemukan isi berita"

                    if len(isi) > 50: # Menaikkan batas minimum panjang berita
                        data_republika.append({
                            'judul': judul,
                            'berita': isi,
                            'url': url,
                        })
                        print(f"    [{len(data_republika)}] Berhasil mengambil: {judul}")

                    if len(data_republika) >= n_berita:
                        break
                    
                    time.sleep(0.5) # Delay antar request detail

                except Exception as e:
                    print(f"    ⚠️ Error saat memproses artikel {url}: {e}")
                    continue
            
            page += 1
            if len(data_republika) >= n_berita:
                break

        # Logika penyimpanan file berada di dalam loop keyword
        if data_republika:
            filename = os.path.join(save_dir, f"republika_news_{keyword.lower().replace(' ','_')}_{time.strftime('%Y%m%d')}.csv")
            with open(filename, "w", newline="", encoding="utf-8-sig") as f:
                writer = csv.DictWriter(f, fieldnames=["judul", "berita", "url"])
                writer.writeheader()
                writer.writerows(data_republika)
            print(f"\n✅ Selesai! {len(data_republika)} artikel '{keyword}' disimpan ke {filename}")
        else:
            print(f"❌ Tidak ada data yang ditemukan untuk '{keyword}'")


In [ ]:
keywords = [
    "BI rate",
    "Pertumbuhan ekonomi",
    "Inflasi",
    "Bank Indonesia",
    "Perbankan digital",
    "IHSG",
    "Pelemahan Rupiah",
    "Investasi",
    "Ekonomi Indonesia",
    "Danantara",
    "OJK",
    "Reksadana",
    "Bursa Efek Indonesia",
    "BEI",
    "Ekonomi digital"
]

republika_scraper(keywords, n_berita=100)


In [ ]:
republika_scraper(keyword="IHSG", n_berita=300)